# Introduction

In this program, users can put the text they would like to analyze into the variable input_text, and the program would output whether this text is offensive language or hate speech or neither of them, along with the category of this text is it is offensive language or hate speech.

# Input Text

In [ ]:
# input text
# change to the input text needs to be analyzed
# this example is a twitter style text on sports with a short offensive language does not have any specific topic
input_text = "Las Vegas Aces win their first WNBA title! Chelsea Gray MVP First pro sports champ for Vegas! get to hit any bitch he wants to"

# Train Model<br>

In this part, we would use the best model architecture of the hate speech detection and topic classfication through the experiments we did. Since the performance has already been evaluated during previous experiments (see seperated file for each model experiments and their evaluation), we would use the whole dataset for training instead of train-test-split.

In [ ]:
import numpy as np
import keras as kr
import pandas as pd

import tensorflow as tf
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPool1DGRU

from wordcloud import WordCloud

from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
import transformers

In [ ]:
import pandas as pd
import numpy as np
import os
#visualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#consistent sized plot
from pylab import rcParams
rcParams['figure.figsize']=12,5
rcParams['axes.labelsize']=12
rcParams['xtick.labelsize']=12
rcParams['ytick.labelsize']=12

In [ ]:
import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
#text preprocessing libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import re
pd.options.display.max_columns = None

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#variable to record the output from two models
hate_speech_classified_output = None
text_category_output = None

In [ ]:
# train hate speech dection model
tweet = pd.read_csv('hate_speech_labeled_data.csv',delimiter=',',engine='python',encoding='utf-8-sig')
c=tweet['class']
tweet.rename(columns={'tweet' : 'text',
                   'class' : 'category'},
                    inplace=True)
a=tweet['text']
b=tweet['category'].map({0: 'hate_speech', 1: 'offensive_language',2: 'neither'})

df= pd.concat([a,b,c], axis=1)
df.rename(columns={'class' : 'label'},
                    inplace=True)

# Handle Diacritics with text normalization
def simplify(text):
    '''Function to handle the diacritics in the text'''
    import unicodedata
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

df['text'] = df['text'].apply(simplify)
# Remove user handles
df['text'].replace(r'@\w+','',regex=True,inplace=True)
# Remove the urls
df['text'].replace(r'http\S+','',regex=True,inplace=True)
tokenizer = TweetTokenizer(preserve_case=True)
df['text'] = df['text'].apply(tokenizer.tokenize)
# Remove Stopwords
stop_words = stopwords.words('english')

#add additional stop words to be removed from the text
additional_list = ['amp','rt','u',"can't",'ur']

for words in additional_list:
    stop_words.append(words)

#remove stop words
def remove_stopwords(text):
    '''Function to remove the stop words from the text corpus'''
    clean_text = [word for word in text if not word in stop_words]
    return clean_text
df['text'] = df['text'].apply(remove_stopwords)

# Spelling corrections
from textblob import TextBlob

def spell_check(text):
    '''Function to do spelling correction using '''
    txtblob = TextBlob(text)
    corrected_text = txtblob.correct()
    return corrected_text

# remove hashtags
def remove_hashsymbols(text):
    pattern = re.compile(r'#')
    text = ' '.join(text)
    clean_text = re.sub(pattern,'',text)
    return tokenizer.tokenize(clean_text)
df['text'] = df['text'].apply(remove_hashsymbols)

# remove short words
def rem_shortwords(text):
    lengths = [1, 2]
    if not isinstance(text, str):
        text = str(text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if len(word) not in lengths]
    new_text = ' '.join(filtered_tokens)
    return new_text
df['text'] = df['text'].apply(rem_shortwords)

# tokenization
df['text'] = df['text'].apply(tokenizer.tokenize)

# Remove digits
def rem_digits(text):
    '''Function to remove the digits from the list of strings'''
    no_digits = []
    for word in text:
        no_digits.append(re.sub(r'\d','',word))
    return ' '.join(no_digits)
df['text'] = df['text'].apply(rem_digits)
df['text'] = df['text'].apply(tokenizer.tokenize)

# Remove special characters
def rem_nonalpha(text):
    '''Function to remove the non-alphanumeric characters from the text'''
    text = [word for word in text if word.isalpha()]
    return text

# remove the non alpha numeric characters from the tweet tokens
df['text'] = df['text'].apply(rem_nonalpha)

# TfidfVectorizer
df1 = df.copy()
df1['text'] = df1['text'].apply(lambda x: ' '.join(x))
X_train = df1['text']
y_train = df1['label']
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
X_train_vec = vectorizer.fit_transform(X_train)

hate_speech_classifier = LogisticRegression()
hate_speech_classifier.fit(X_train_vec,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
bert_base_model = transformers.TFAutoModel.from_pretrained('bert-large-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    print(df.head())
    return df
def tokenize_texts(text, tokenizer, max_len):
    text = text.tolist()
    tokenized_output = tokenizer(
        text = text,
        padding='max_length',
        max_length=max_len,
        truncation=True,
        return_tensors='np',
        is_split_into_words=False,
        stride = 0,
        return_overflowing_tokens=False,
    )
    tokenized_input_ids = tokenized_output['input_ids']
    return tokenized_input_ids
def split_data(df, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(df['article'], df['category'], test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test
def one_hot_encode(labels, num_classes=10):
    one_hot_encoded_label = tf.keras.utils.to_categorical(labels, num_classes=num_classes, dtype='int32')
    return one_hot_encoded_label
df = load_and_preprocess_data('data_preprocessed.csv')

X_train, X_test, y_train, y_test = split_data(df, test_size=0.2, random_state=42)

max_len = 60
tokenizer_x_train = tokenizer(
        text = X_train.astype('str').tolist(),
        padding='max_length',
        max_length=max_len,
        truncation=True,
        return_tensors='np',
        is_split_into_words=False,
        stride = 0,
        return_overflowing_tokens=False,
    )
X_train_tokenized = tokenizer_x_train['input_ids']

tokenizer_x_test = tokenizer(
        text = X_test.astype('str').tolist(),
        padding='max_length',
        max_length=max_len,
        truncation=True,
        return_tensors='np',
        is_split_into_words=False,
        stride = 0,
        return_overflowing_tokens=False,
    )
X_test_tokenized = tokenizer_x_test['input_ids']

encoder = LabelEncoder()
y_train = one_hot_encode(encoder.fit_transform(y_train))
y_test = one_hot_encode(encoder.transform(y_test))


def construct_bert_based_classifier(bert_layer, total_classes=10):

    bert_input = tf.keras.layers.Input(shape=(60,), dtype=tf.int32, name='bert_input_layer')
    dropout_seq = tf.keras.layers.Dropout(0.3, name='dropout_for_generalization')(bert_layer(bert_input)[0][:, 0, :])
    prediction_layer = tf.keras.layers.Dense(total_classes, activation='softmax', name='prediction_layer')(dropout_seq)
    bert_classifier = tf.keras.Model(inputs=bert_input, outputs=prediction_layer)
    bert_classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])

    return bert_classifier

text_classification_model = construct_bert_based_classifier(bert_base_model)

text_classification_model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

        category                                            article
0         COMEDY  of the funniest tweets about cats and dogs thi...
1      PARENTING  the funniest tweets from parents this week sep...
2         SPORTS  maury wills basestealing shortstop for dodgers...
3  ENTERTAINMENT  golden globes returning to nbc in january afte...
4       POLITICS  biden says us forces would defend taiwan if ch...
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 60)]              0         
                                                                 
 tf_bert_model_3 (TFBertMod  TFBaseModelOutputWithPo   335141888 
 el)                         olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 60, 1024),                       
                              pooler_output=(None

In [ ]:

BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE

train_data = tf.data.Dataset.from_tensor_slices((X_train_tokenized, y_train)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO)
test_data = tf.data.Dataset.from_tensor_slices((X_test_tokenized, y_test)).batch(BATCH_SIZE)

steps_per_epoch = 300
training_history = text_classification_model.fit(train_data, steps_per_epoch=steps_per_epoch, epochs=5)


Epoch 1/5
300/300 [==============================] - 369s 1s/step - loss: 0.6905 - accuracy: 0.7860
Epoch 2/5
300/300 [==============================] - 368s 1s/step - loss: 0.5857 - accuracy: 0.8172
Epoch 3/5
300/300 [==============================] - 368s 1s/step - loss: 0.5407 - accuracy: 0.8304
Epoch 4/5
300/300 [==============================] - 367s 1s/step - loss: 0.5290 - accuracy: 0.8306
Epoch 5/5
300/300 [==============================] - 368s 1s/step - loss: 0.5014 - accuracy: 0.8382


# Generate output

In [ ]:
# go through hate speech dection model
def hate_speech_preprocess(input_text):
  input_text = simplify(input_text)
  input_text.replace(r'@\w+','')
  input_text.replace(r'http\S+','')
  tokenizer = TweetTokenizer(preserve_case=True)
  input_text = tokenizer.tokenize(input_text)
  input_text = remove_stopwords(input_text)
  input_text = remove_hashsymbols(input_text)
  input_text = rem_shortwords(input_text)
  input_text = tokenizer.tokenize(input_text)
  input_text = rem_digits(input_text)
  input_text = tokenizer.tokenize(input_text)
  input_text = rem_nonalpha(input_text)
  input_text = ' '.join(input_text)
  input_text = vectorizer.transform([input_text])
  input_text_hate_speech = input_text
  return input_text_hate_speech
input_text_hate_speech = hate_speech_preprocess(input_text)
hate_speech_classified_output = hate_speech_classifier.predict(input_text_hate_speech)
if hate_speech_classified_output == 0:
    print('This is a hate speech')
elif hate_speech_classified_output == 1:
    print('This is an offensive language')
elif hate_speech_classified_output==2:
    print('This is not a hate speech or offensive language')
else:
    print("error in hate speech detection")

This is an offensive language


In [ ]:
# go through the text classification model
def text_classification_preprocess(input_text):
    input_text = simplify(input_text)
    max_len = 60
    tokenizer_x_train = tokenizer(
            text = input_text,
            padding='max_length',
            max_length=max_len,
            truncation=True,
            return_tensors='np',
            is_split_into_words=False,
            stride = 0,
            return_overflowing_tokens=False,
        )
    X_train_tokenized = tokenizer_x_train['input_ids']
    return X_train_tokenized

input_text_classification_preprocessed = text_classification_preprocess(input_text)
y_pred = text_classification_model.predict(input_text_classification_preprocessed)
predicted_category = encoder.classes_[np.argmax(y_pred, axis=1)]
text_category_output = predicted_category[0]

1/1 [==============================] - 0s 118ms/step


In [ ]:
if hate_speech_classified_output == 2:
  print('This is not a hate speech or offensive language')
elif hate_speech_classified_output == 0:
  print('This is a hate speech on ', text_category_output)
elif hate_speech_classified_output == 1:
  print('This is an offensive language on', text_category_output.lower())
else:
  print("error in hate speech detection")


This is an offensive language on sports
